# Assignment 2: Milestone I Natural Language Processing
## Task 1. Basic Text Pre-processing
#### Student Name: Pratham Radhakrishna
#### Student ID: S3997064

## Introduction
Objective
The objective of Task 1 is to perform basic text pre-processing on a dataset of clothing reviews. The focus is on cleaning and preparing the "Review Text" column for further analysis and model building. This task involves several steps to ensure the text data is in a suitable format for feature extraction and machine learning.

Steps Involved
Extract Information About the Review:

Load the dataset and examine its structure.
Identify and handle missing values in the "Review Text" column.
Tokenization:

Tokenize each clothing review using the specified regular expression r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?".
Tokenization involves splitting the text into individual words or tokens.
Convert to Lowercase:

Convert all words to lowercase to ensure uniformity and reduce redundancy.
Remove Short Words:

Remove words with a length less than 2 characters, as they are often not meaningful.
Remove Stopwords:

Remove common stopwords using a provided stop words list (stopwords_en.txt).
Stopwords are words that are frequently used but carry little informational value (e.g., "and", "the", "is").
Remove Words that Appear Only Once:

Identify and remove words that appear only once in the entire document collection, as they may not contribute significantly to the analysis.
Remove the Top 20 Most Frequent Words:

Calculate the document frequency for each word.
Identify and remove the top 20 most frequent words to reduce noise and focus on more informative words.
Save the Processed Data:

Save the cleaned and processed review text to a new CSV file (processed.csv).
Build a Vocabulary:

Create a vocabulary of the cleaned/processed reviews.
Save the vocabulary to a text file (vocab.txt) with each word and its corresponding index.



## Importing libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd

import re
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import nltk
from collections import Counter
nltk.download('stopwords')
nltk.download('punkt')
from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.1 Examining and loading data
- Examine the data and explain your findings
- Load the data into proper data structures and get it ready for processing.

In [2]:
# Code to inspect the provided data file...

df = pd.read_csv('assignment3.csv')
df.head()
#**Data Overview**:
#   - The dataset contains 19662 rows and 10 columns.
#   - The columns include various attributes of clothing reviews such as 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', etc.

Clothing ID  Age                    Title  \
0         1077   60  Some major design flaws   
1         1049   50         My favorite buy!   
2          847   47         Flattering shirt   
3         1080   49  Not for the very petite   
4          858   39     Cagrcoal shimmer fun   

                                         Review Text  Rating  Recommended IND  \
0  I had such high hopes for this dress and reall...       3                0   
1  I love, love, love this jumpsuit. it's fun, fl...       5                1   
2  This shirt is very flattering to all due to th...       5                1   
3  I love tracy reese dresses, but this one is no...       2                0   
4  I aded this in my basket at hte last mintue to...       5                1   

   Positive Feedback Count   Division Name Department Name Class Name  
0                        0         General         Dresses    Dresses  
1                        0  General Petite         Bottoms      Pants  
2                        6         General            Tops    Blouses  
3                        4         General         Dresses    Dresses  
4                        1  General Petite            Tops      Knits

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19662 entries, 0 to 19661
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              19662 non-null  int64 
 1   Age                      19662 non-null  int64 
 2   Title                    19662 non-null  object
 3   Review Text              19662 non-null  object
 4   Rating                   19662 non-null  int64 
 5   Recommended IND          19662 non-null  int64 
 6   Positive Feedback Count  19662 non-null  int64 
 7   Division Name            19662 non-null  object
 8   Department Name          19662 non-null  object
 9   Class Name               19662 non-null  object
dtypes: int64(5), object(5)
memory usage: 1.5+ MB


In [4]:
print("\nSummary statistics of the dataset:")
print(df.describe())


Summary statistics of the dataset:
        Clothing ID           Age        Rating  Recommended IND  \
count  19662.000000  19662.000000  19662.000000     19662.000000   
mean     921.297274     43.260808      4.183145         0.818177   
std      200.227528     12.258122      1.112224         0.385708   
min        1.000000     18.000000      1.000000         0.000000   
25%      861.000000     34.000000      4.000000         1.000000   
50%      936.000000     41.000000      5.000000         1.000000   
75%     1078.000000     52.000000      5.000000         1.000000   
max     1205.000000     99.000000      5.000000         1.000000   

       Positive Feedback Count  
count             19662.000000  
mean                  2.652477  
std                   5.834285  
min                   0.000000  
25%                   0.000000  
50%                   1.000000  
75%                   3.000000  
max                 122.000000  


In [5]:
# Check for missing values
print("\nMissing values in the dataset:")
print(df.isnull().sum())
# no null values in any of the columns 


Missing values in the dataset:
Clothing ID                0
Age                        0
Title                      0
Review Text                0
Rating                     0
Recommended IND            0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
dtype: int64


In [6]:
# Displaying unique values in 'Recommended IND' column to understand the labels
print("\nUnique values in 'Recommended IND' column:")
print(df['Recommended IND'].unique())


Unique values in 'Recommended IND' column:
[0 1]


In [7]:
 #Displaying the distribution of 'Recommended IND' column
print("\nDistribution of 'Recommended IND' column:")
print(df['Recommended IND'].value_counts())


Distribution of 'Recommended IND' column:
1    16087
0     3575
Name: Recommended IND, dtype: int64


### 1.2 Pre-processing data


In [8]:
# Loading the stopwords from file
stopwords_file = 'stopwords_en.txt'  
with open(stopwords_file, 'r') as file:
    stop_words = set(file.read().splitlines())

In [9]:
#Step 2: Tokenize Each Clothing Review
#Using RegexpTokenizer to tokenize words based on the specified regular expression.

# Defining the tokenization function using RegexpTokenizer
def tokenize(text):
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
    tokens = tokenizer.tokenize(text)
    return tokens

In [10]:
# Applying tokenization to the 'Review Text' column
df['Tokenized_Review'] = df['Review Text'].apply(lambda x: tokenize(x) if pd.notnull(x) else [])

In [11]:
#Step 3: Converting All Words to Lowercase and Remove Words with a Length Less than 2
# Defineing the function to process tokens: convert to lowercase and remove short words
def process_tokens(tokens):
    # Converting to lowercase and remove words with length less than 2
    processed_tokens = [word.lower() for word in tokens if len(word) >= 2]
    return processed_tokens

# Applying lowercase conversion and short word removal
df['Processed_Review'] = df['Tokenized_Review'].apply(process_tokens)

In [12]:
#Step 4: Removing Stopwords
# Defineing the function to remove stopwords
def remove_stopwords(tokens):
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

# Applying stop words removal
df['Processed_Review'] = df['Processed_Review'].apply(remove_stopwords)

In [13]:
#Step 5: Removing Words that Appear Only Once in the Document Collection
# Flatten the list of all tokens
all_tokens = [word for tokens in df['Processed_Review'] for word in tokens]

# Calculating the frequency of each word
word_freq = Counter(all_tokens)

# Removing words that appear only once
df['Processed_Review'] = df['Processed_Review'].apply(lambda tokens: [word for word in tokens if word_freq[word] > 1])

In [14]:

# Calculate document frequency for each word using FreqDist
doc_freq = FreqDist()
for tokens in df['Processed_Review']:
    doc_freq.update(set(tokens))

# Get the top 20 most frequent words
most_frequent_words = set([word for word, freq in doc_freq.most_common(20)])

# Remove the top 20 most frequent words from the processed reviews
df['Processed_Review'] = df['Processed_Review'].apply(lambda tokens: [word for word in tokens if word not in most_frequent_words])


In [15]:
#Step 7: Save the Processed Data as processed.csv
# Joining the processed tokens back into a single string for each review
df['Processed_Review_Text'] = df['Processed_Review'].apply(lambda tokens: ' '.join(tokens))

# Save the processed data to a CSV file
df.to_csv('processed.csv', index=False)

## Saving required outputs
Save the requested information as per specification.
- vocab.txt

In [16]:
# code to save output data...
#Step 8: Building a Vocabulary of the Cleaned/Processed Reviews and Save it in a vocab.txt File
# Build the vocabulary
vocabulary = sorted(set(word for tokens in df['Processed_Review'] for word in tokens))

# Create a dictionary with word indices
vocab_dict = {word: idx for idx, word in enumerate(vocabulary)}

# Save the vocabulary to a txt file
with open('vocab.txt', 'w') as file:
    for word, idx in vocab_dict.items():
        file.write(f"{word}:{idx}\n")

## Summary
The objective of Task 1 is to clean and prepare the "Review Text" from a dataset of clothing reviews for further analysis. The steps include loading the dataset, handling missing values, tokenizing the text using a specified regular expression, converting words to lowercase, and removing words with fewer than 2 characters. Additionally, common stopwords are removed using a provided list, and words that appear only once or are among the top 20 most frequent are filtered out. The cleaned review text is saved to processed.csv, and a vocabulary of the processed text is built and saved to vocab.txt. This results in a dataset ready for feature extraction and machine learning, along with a corresponding vocabulary file.

In [17]:
df

Clothing ID  Age                                              Title  \
0             1077   60                            Some major design flaws   
1             1049   50                                   My favorite buy!   
2              847   47                                   Flattering shirt   
3             1080   49                            Not for the very petite   
4              858   39                               Cagrcoal shimmer fun   
...            ...  ...                                                ...   
19657         1104   34                     Great dress for many occasions   
19658          862   48                         Wish it was made of cotton   
19659         1104   31                              Cute, but see through   
19660         1084   28  Very cute dress, perfect for summer parties an...   
19661         1104   52                    Please make more like this one!   

                                             Review Text  Rating  \
0      I had such high hopes for this dress and reall...       3   
1      I love, love, love this jumpsuit. it's fun, fl...       5   
2      This shirt is very flattering to all due to th...       5   
3      I love tracy reese dresses, but this one is no...       2   
4      I aded this in my basket at hte last mintue to...       5   
...                                                  ...     ...   
19657  I was very happy to snag this dress at such a ...       5   
19658  It reminds me of maternity clothes. soft, stre...       3   
19659  This fit well, but the top was very see throug...       3   
19660  I bought this dress for a wedding i have this ...       3   
19661  This dress in a lovely platinum is feminine an...       5   

       Recommended IND  Positive Feedback Count   Division Name  \
0                    0                        0         General   
1                    1                        0  General Petite   
2                    1                        6         General   
3                    0                        4         General   
4                    1                        1  General Petite   
...                ...                      ...             ...   
19657                1                        0  General Petite   
19658                1                        0  General Petite   
19659                0                        1  General Petite   
19660                1                        2         General   
19661                1                       22  General Petite   

      Department Name Class Name  \
0             Dresses    Dresses   
1             Bottoms      Pants   
2                Tops    Blouses   
3             Dresses    Dresses   
4                Tops      Knits   
...               ...        ...   
19657         Dresses    Dresses   
19658            Tops      Knits   
19659         Dresses    Dresses   
19660         Dresses    Dresses   
19661         Dresses    Dresses   

                                        Tokenized_Review  \
0      [I, had, such, high, hopes, for, this, dress, ...   
1      [I, love, love, love, this, jumpsuit, it's, fu...   
2      [This, shirt, is, very, flattering, to, all, d...   
3      [I, love, tracy, reese, dresses, but, this, on...   
4      [I, aded, this, in, my, basket, at, hte, last,...   
...                                                  ...   
19657  [I, was, very, happy, to, snag, this, dress, a...   
19658  [It, reminds, me, of, maternity, clothes, soft...   
19659  [This, fit, well, but, the, top, was, very, se...   
19660  [I, bought, this, dress, for, a, wedding, i, h...   
19661  [This, dress, in, a, lovely, platinum, is, fem...   

                                        Processed_Review  \
0      [high, hopes, wanted, work, initially, petite,...   
1      [jumpsuit, fun, flirty, fabulous, time, compli...   
2      [shirt, due, adjustable, front, tie, length, l...   
3      [tracy, reese, dresses, petite, feet, tall, br.